A.S. Lundervold, v111022

> **Note:** This is a short notebook giving a quick taste of a concept that's also covered elsewhere in the course. It should be regarded as extra material. 

# Setup

In [ ]:
%matplotlib inline

import numpy as np, pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path
import seaborn as sns 
import sklearn
from sklearn import datasets

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

# Data

La oss bruke diabetes-datasettet:

In [ ]:
df = pd.read_csv('https://assets.datacamp.com/production/course_1939/datasets/diabetes.csv')

X, y  = df.drop('diabetes', axis=1), df.diabetes

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
df.head()

In [ ]:
from sklearn.metrics import accuracy_score

# Pipelines

Som vi har erfart er det en rekke ting en pleier å gjøre med et datasett før en trener modeller: imputering, skalering, feature engineering, feature selection, osv. Generelt, en rekke _transformasjoner_ av data. Disse bestemmes typisk fra et treningsdatasett, og anvendes så både på trening- og testsett (tenk: `.fit_transform` på trening, `.transform` på test). 

Når en skriver kode for å gjøre alt dette kan det ofte se veldig uryddig ut, noe som øker sjansen for å gjøre feil. Det kan også bli vanskelig å designe eksperiment der en tester ut ulike varianter av disse oppsettene, for eksempel via et grid search. 

Ved å bruke scikit-learn [Pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline) kan dette struktureres bedre. 

Vi husker for eksempel hvordan vi behandlet datasettet fra innlevering 1 i notebooken om ensembling: 

1. Imputering
2. Skalering
3. Prediksjon med et voting ensemble

I større maskinlæringsprosjekt er det gjerne mange slike steg. 

Her er en måte å bygge dette som en pipeline. 

Vi lager først vårt ensemble:

In [ ]:
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
gb = GradientBoostingClassifier(random_state=42)
svc = SVC(random_state=42)
gnb = GaussianNB()

models = [('rf', rf), 
          ('gb', gb),
          ('svc', svc),
          ('gnb', gnb)]

ensemble = VotingClassifier(models)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

Her er vår pipeline:

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
pipeline = make_pipeline(SimpleImputer(), 
                         MinMaxScaler(), 
                         ensemble)

In [ ]:
pipeline

Denne kan så tilpasses treningsdata og brukes til å produsere prediksjoner på testdata:

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(pipeline, X_train, y_train, cv=3, scoring='accuracy')

## Modifisere pipelines

Med pipelines er det enklere å kjøre eksperimenter med ulike oppsett:

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
for scaler in [MinMaxScaler(), StandardScaler()]:
    pipeline = make_pipeline(SimpleImputer(), scaler, ensemble)
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='accuracy')
    print(f"Cross val scores for {scaler}: {scores}")
    print(f"Mean: {np.mean(scores)}\n")

En kan også bruke GridSearchCV kombinert med pipelines (https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html#sphx-glr-auto-examples-compose-plot-compare-reduction-py)